# BFS

## BFS 기본적인 알고리즘 구현
- `collections.deque` 사용
- graph
![그래프 예시](../images/graph_example00.png)

In [1]:
ex_graph = {
    "A": ["B", "C"],
    "B": ["A", "D", "E"],
    "C": ["A", "F"],
    "D": ["B"],
    "E": ["B", "F"],
    "F": ["C", "E"],
}

In [2]:
from collections import deque


def bfs(graph: dict, start: str) -> list[str]:
    visited = set()  # 방문 기록용 set

    visited.add(start)  # 시작 node 에 방문 표시
    queue = deque([start])  # queue 생성 및 시작 node 추가

    traversal_order = []  # 탐색 순서 리스트

    while queue:
        # node 꺼냄
        node = queue.popleft()

        # node 처리
        traversal_order.append(node)

        # node 의 이웃 처리
        for neighbor in graph[node]:
            # 방문하지 않은 이웃일 때만!
            if neighbor not in visited:
                # 방문 처리 및 queue 에 추가
                visited.add(neighbor)
                queue.append(neighbor)
    return traversal_order

In [3]:
print("traversal order:", bfs(ex_graph, "A"))

traversal order: ['A', 'B', 'C', 'D', 'E', 'F']


## 최단 경로 찾기

- 문제: 주어진 그래프에서 시작 노드 A 에서 목표 노드 F 까지 가는 최단 경로를 찾으세요. 각 간선의 길이는 모두 1로 동일합니다.
- 가중치가 없는 그래프에서만 bfs 로 최단 경로 찾을 수 있음
- 가중치가 있는 그래프는 다익스트라 알고리즘이나 벨만포드 알고리즘을 써야 함

In [4]:
def find_shortest_path(graph: dict, start: str, end: str) -> list[str]:
    if start not in graph or end not in graph:
        return []

    if start == end:
        return [start]

    visited = {start}
    # queue 에 node 와 이 node 까지 온 경로를 저장
    queue = deque([(start, [start])])  # (node, 경로)

    while queue:
        node, path = queue.popleft()

        for neighbor in graph.get(node, []):
            if neighbor == end:  # 목표 도달!
                return [*path, neighbor]

            if neighbor not in visited:
                visited.add(neighbor)
                # 현재까지의 경로에 이웃을 추가
                queue.append((neighbor, [*path, neighbor]))
    # 경로를 찾을 수 없음
    return []


print("A 에서 F 까지 최단 경로:", find_shortest_path(ex_graph, "A", "F"))

A 에서 F 까지 최단 경로: ['A', 'C', 'F']


In [5]:
def find_shortest_distance(graph: dict, start: str, end: str) -> int:
    if start not in graph or end not in graph:
        return 0

    if start == end:
        return 0

    distances = {start: 0}  # 방문 node 와 그 node 까지의 거리를 저장
    queue = deque([start])

    while queue:
        node = queue.popleft()
        current_distance = distances.get(node, 0)

        for neighbor in graph.get(node, []):
            if neighbor == end:  # 목표 도달!
                return current_distance + 1

            if neighbor not in distances:
                distances[neighbor] = current_distance + 1
                queue.append(neighbor)
    return -1

In [6]:
print("A 에서 F 까지의 최단거리:", find_shortest_distance(ex_graph, "A", "F"))

A 에서 F 까지의 최단거리: 2


## 미로에서 최단 경로 찾기
![maze](../images/graph_maze.png)

```
0 1 0 0 0
0 1 0 1 0
0 0 0 1 0
1 1 0 1 0
0 0 0 0 0
```

In [7]:
from collections import defaultdict


input_maze = [
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
]


# 1. input_maze 를 인접 리스트로 만들기
#     - 행과 열의 index 즉, 좌표가 graph 의 node
#     - 이웃은 상, 화, 좌, 우 이지만, 값이 0 인 것들
def convert_maze_to_graph(maze: list[list[int]]) -> defaultdict:
    graph = defaultdict(list)
    for i, row in enumerate(maze):
        for j, _ in enumerate(row):
            node = (i, j)
            neighbors = []
            for direction in [(0, 1), (0, -1), (1, 0), (-1, 0)]:  # 이웃 찾기
                # 이웃 노드
                neighbor = (i + direction[0], j + direction[1])
                if neighbor[0] < 0 or neighbor[0] >= len(maze):  # 좌표 밖이라서 무시
                    continue

                if neighbor[1] < 0 or neighbor[1] >= len(maze[0]):  # 좌표 밖이라서 무시
                    continue

                value = maze[neighbor[0]][neighbor[1]]
                if value == 0:  # value 가 0 이면 진짜 이웃, 1 이면 이웃이 아님
                    neighbors.append(neighbor)
            graph[node] = neighbors
    return graph


# 2. 기존의 bfs 적용
def find_shortest_path_in_maze(maze_graph: defaultdict, start: tuple[int, int], end: tuple[int, int]) -> list:
    visited = set(start)
    queue = deque([(start, [start])])

    while queue:
        node, path = queue.popleft()

        for neighbor in maze_graph.get(node, []):
            if neighbor == end:
                return [*path, neighbor]

            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, [*path, neighbor]))
    return []


graph_maze = convert_maze_to_graph(input_maze)
print("(0, 0) -> (4, 4):", find_shortest_path_in_maze(graph_maze, (0, 0), (4, 4)))

(0, 0) -> (4, 4): [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]


In [8]:
def solve_maze(maze: list[list[int]], start: tuple[int, int], end: tuple[int, int]) -> list:
    len_row, len_col = len(maze), len(maze[0])
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # 상, 하, 좌, 우

    visited = set(start)
    queue = deque([(start, [start])])  # node 와, node 까지의 경로
    while queue:
        (row, col), path = queue.popleft()
        if (row, col) == end:
            return path

        for direction in directions:  # 내 이웃을 순회하는 것!
            neighbor = (row + direction[0], col + direction[1])
            if neighbor[0] < 0 or len_row <= neighbor[0]:
                continue  # 좌표 밖은 무시

            if neighbor[1] < 0 or len_col <= neighbor[1]:
                continue  # 좌표 밖은 무시

            if maze[neighbor[0]][neighbor[1]] == 1:
                continue  # 연결되지 않은 노드!

            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, [*path, neighbor]))
    return []  # 경로 없음


print("(0, 0) -> (4, 4):", solve_maze(input_maze, (0, 0), (4, 4)))

(0, 0) -> (4, 4): [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]


In [9]:
def solve_maze_with_visualization(maze: list[list[int]], start: tuple[int, int], end: tuple[int, int]) -> list:
    len_row, len_col = len(maze), len(maze[0])
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]

    visited = set(start)
    queue = deque([(start, [start])])

    while queue:
        node, path = queue.popleft()
        if node == end:
            # 경로 찾았으므로, 경로 보여주고(1), 그 경로 반환(2)

            visual_board = [row[:] for row in maze]  # 보여주기 위해서 미로판을 복사
            visual_board[start[0]][start[1]] = "S"  # 시작점 표시
            visual_board[end[0]][end[1]] = "E"  # 도착점 표시

            for neighbor in path[1:]:
                visual_board[neighbor[0]][neighbor[1]] = "*"

            # 시각화를 위한 보드판 출력
            for visual_row in visual_board:
                print(" ".join([str(cell) for cell in visual_row]))

            return path

        for direction in directions:
            neighbor = (node[0] + direction[0], node[1] + direction[1])

            if neighbor[0] < 0 or len_row <= neighbor[0]:
                continue

            if neighbor[1] < 0 or len_col <= neighbor[1]:
                continue

            if maze[neighbor[0]][neighbor[1]] == 0:
                visited.add(neighbor)
                queue.append((neighbor, [*path, neighbor]))
    return []


print("경로:", solve_maze_with_visualization(input_maze, (0, 0), (4, 4)))

S 1 0 0 0
* 1 0 1 0
* * * 1 0
1 1 * 1 0
0 0 * * *
경로: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]


## 연결 요소 찾기

![graph 예시](../images/graph_example01.png)




In [10]:
new_graph = {
    "A": ["B"],
    "B": ["A", "C", "D"],
    "C": ["B", "D"],
    "D": ["B", "C"],
    "E": ["F", "G"],
    "F": ["E"],
    "G": ["E"],
    "H": [],
}

In [11]:
def find_connected_components(graph: dict):
    components = []
    visited = set()

    for node in graph:
        if node not in visited:  # 첫 방문 즉, 새로운 component 발견
            component = []

            # 여기서 부터는 기존의 bfs 와 같음
            queue = deque([node])
            visited.add(node)
            while queue:
                current = queue.popleft()

                component.append(current)  # node 처리

                for neighbor in graph.get(current, []):
                    if neighbor not in visited:
                        visited.add(neighbor)
                        queue.append(neighbor)
            components.append(component)
    return components


find_connected_components(new_graph)

[['A', 'B', 'C', 'D'], ['E', 'F', 'G'], ['H']]

## 이분 그래프(Bipartite Graph) 판별

![이분 그래프](../images/graph_example02.png)

In [12]:
ex_graph02 = {
    "A": ["C"],
    "B": ["D"],
    "C": ["A", "E", "H"],
    "D": ["B", "F"],
    "E": ["C", "G"],
    "F": ["D"],
    "G": ["E", "H"],
    "H": ["C", "G"],
}

In [13]:
def is_bipartite(graph: dict) -> tuple[bool, set | None, set | None]:
    """
    그래프의 모든 node 를 두 가지 색으로 칠할 수 있어야한다는
    조건을 바탕으로 만든 판별 코드
    """
    # 색 집합이면서, 방문 체크
    colors = {}

    for start_node in graph:
        if start_node not in colors:
            colors[start_node] = 1  # 방문 표시 및 속하는 집합 번호 표시

            # bfs 시작
            queue = deque([start_node])
            while queue:
                node = queue.popleft()
                current_color = colors[node]

                # 이웃 처리
                for neighbor in graph.get(node, []):
                    if neighbor not in colors:  # 방문하지 않았다면,
                        # 현재 색과 부호를 반대로 함으로써, 다른 집합임을 표현
                        colors[neighbor] = -current_color  # 더불어 방문 표시
                        queue.append(neighbor)
                    elif colors[neighbor] == current_color:
                        # 방문한 적이 있어 집합이 설정이 되었는데,
                        # 그 집합이 현재 집합과 같다!
                        # => 인접한 node 가 같은 색상이므로, 이분 그래프가 아니다.
                        return False, None, None
    set1 = set()
    set2 = set()
    for node, color in colors.items():
        if color == 1:
            set1.add(node)
        else:
            set2.add(node)
    return True, set1, set2


is_bipartite(ex_graph02)

(True, {'A', 'B', 'E', 'F', 'H'}, {'C', 'D', 'G'})

## 레벨 별 순회(Level Order Traversal)

![그래프](../images/graph_example02.png)

In [14]:
ex_graph03 = {
    "A": ["C"],
    "B": ["D"],
    "C": ["A", "E", "H"],
    "D": ["B", "F"],
    "E": ["C", "G"],
    "F": ["D"],
    "G": ["E", "H"],
    "H": ["C", "G"],
}

In [15]:
def traverse_on_level_order(graph: dict, start: str) -> list:
    result = []

    # 기본 bfs 와 같은데, 레벨만 같이 넣어준다.
    visited = set(start)
    queue = deque([(start, 0)])  # root node 는 level 이 0

    while queue:
        node, level = queue.popleft()

        # 새 레벨 시작
        if level == len(result):
            result.append([])  # 새 레벨 그룹용 list 추가

        # 현재 노드를 레벨 그룹 list 에 추가
        result[level].append(node)

        # 이제 이웃 처리
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, level + 1))

    return result


traverse_on_level_order(ex_graph03, "A")

[['A'], ['C'], ['E', 'H'], ['G']]

# DFS

## 기본 DFS 구현

![graph example](../images/graph_example03.png)

### 스택 기반 DFS

In [16]:
ex_graph04 = {
    "A": ["B", "C"],
    "B": ["D", "E"],
    "C": ["F", "G"],
}

In [17]:
def dfs_with_stack(graph: dict, start: str) -> list[str]:
    stack = deque([start])

    visited = set()
    traversal_order = []
    while stack:
        node = stack.pop()

        if node not in visited:
            visited.add(node)
            traversal_order.append(node)

            # 항상 알파벳 순으로 방문하도록 sorted(..,reverse=) 추가
            neighbors = sorted(graph.get(node, []), reverse=True)
            for neighbor in neighbors:
                if neighbor not in visited:
                    stack.append(neighbor)

    return traversal_order


print("stack 기반 dfs 로 탐색(A부터):", dfs_with_stack(ex_graph04, "A"))

stack 기반 dfs 로 탐색(A부터): ['A', 'B', 'D', 'E', 'C', 'F', 'G']


### 재귀 기반 DFS

In [18]:
def def_with_recursion(graph: dict, start: str) -> list[str]:
    visited = set()
    traversal_order = []

    def dfs_helper(node):
        visited.add(node)
        traversal_order.append(node)

        # 항상 알파벳 순으로 방문하도록 sorted(..) 추가
        neighbors = sorted(graph.get(node, []))
        for neighbor in neighbors:
            if neighbor not in visited:
                dfs_helper(neighbor)

    dfs_helper(start)  # 재귀적으로 탐색 시작

    return traversal_order


print("recursion 기반 dfs 로 탐색(A부터):", def_with_recursion(ex_graph04, "A"))

recursion 기반 dfs 로 탐색(A부터): ['A', 'B', 'D', 'E', 'C', 'F', 'G']


## 경로 존재 확인 (Path Finding)

- BFS 로도 구현 가능
- 실제로는 휴리스틱 A* 알고리즘을 사용

![graph ex04](../images/graph_example04.png)

In [19]:
ex_graph05 = {
    "A": ["B"],
    "B": ["D", "E"],
    "C": ["F", "G"],
    "E": ["H"],
}

### with Stack

In [20]:
def has_path_with_stack(graph: dict, start: str, end: str) -> bool:
    if start == end:
        return True

    visited = set()
    stack = deque([start])

    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            if node == end:  # 경로 찾음!
                return True

            for neighbor in graph.get(node, []):
                if neighbor not in visited:
                    stack.append(neighbor)
    # 경로 없음
    return False


print("A 와 H 사이에 경로가 있나?", has_path_with_stack(ex_graph05, "A", "H"))
print("A 와 G 사이에 경로가 있나?", has_path_with_stack(ex_graph05, "A", "G"))

A 와 H 사이에 경로가 있나? True
A 와 G 사이에 경로가 있나? False


with Recursion

In [21]:
def has_path_with_recursion(graph: dict, start: str, end: str) -> bool:
    if start == end:
        return True

    visited = set()

    def dfs_helper(current: str) -> bool:
        # 방문 처리
        visited.add(current)

        # 도달 했는지 확인
        if current == end:
            return True

        # 도달 못 했으므로, 이웃 node 확인
        for neighbor in graph.get(current, []):
            if neighbor not in visited:
                # 방문하지 않은 이웃 node 에 대해서 재귀 함수로 경로 있는지 확인
                return dfs_helper(neighbor)
        return False

    return dfs_helper(start)


print("A 와 D 사이에 경로가 있나?", has_path_with_recursion(ex_graph05, "A", "D"))
print("A 와 C 사이에 경로가 있나?", has_path_with_recursion(ex_graph05, "A", "C"))

A 와 D 사이에 경로가 있나? True
A 와 C 사이에 경로가 있나? False
